In [ ]:
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Feature matrix.csv')
print(df.head(5))

   id  label     avg_X  max_X  min_X  X_median     avg_Y  max_Y  min_Y  \
0  15    0.0  19.21875    7.0    1.0      19.0 -42.50625  -64.0  -27.0   
1  15    0.0  17.93750   22.0   15.0      18.0 -43.73750  -45.0  -41.0   
2  15    0.0 -27.60625  -37.0  -20.0     -27.0   9.96250    9.0   -1.0   
3  15    0.0 -28.59375  -38.0  -17.0     -29.0  10.63750    9.0   -1.0   
4  15    0.0 -50.07500  -99.0  -13.0     -48.0 -29.84375    6.0   -1.0   

   Y_median  ...   min_EDA  EDA_median   avg_HR  max_HR  min_HR  HR_median  \
0     -43.0  ...  3.801695    3.976439  103.190  103.27  103.07     103.20   
1     -44.0  ...  3.547946    3.619714  103.188  103.30  103.00     103.25   
2      10.0  ...  4.737687    4.808173  109.362  110.00  108.68     109.38   
3      10.0  ...  4.615938    4.645414  112.220  112.35  111.90     112.28   
4     -29.0  ...  4.038785    4.157969   99.768   99.92   99.62      99.77   

   avg_TEMP  max_TEMP  min_TEMP  TEMP_median  
0    33.876     33.95     33.81        

In [ ]:
class_counts = df['label'].value_counts()
print(class_counts)

2.0    53081
0.0    13551
1.0     4851
Name: label, dtype: int64


XGBoost

In [ ]:
import pandas as pd
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder


# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Prepare features and target variable
X = df.drop(columns=['id', 'label'])  # Exclude 'id' and 'label' columns
y = df['label']
participants = df['id']

# Define oversampler (SMOTE)
oversampler = SMOTE(random_state=42)

# Define undersampler (RandomUnderSampler)
undersampler = RandomUnderSampler(random_state=42)

# Implement leave-one-participant-out cross-validation
logo = LeaveOneGroupOut()

# Lists to store evaluation metrics
accuracies = []
precisions = []
recalls = []
f1_scores = []
conf_matrices = []

fold_number = 1  # Counter for fold number

# XGBoost parameters for tuning
params = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300]
}

best_params = None
best_accuracy = 0

# Iterate over different parameter combinations
for max_depth in params['max_depth']:
    for learning_rate in params['learning_rate']:
        for n_estimators in params['n_estimators']:
            print(f"Training XGBoost with max_depth={max_depth}, learning_rate={learning_rate}, n_estimators={n_estimators}:")

            for train_index, test_index in logo.split(X, y, groups=participants):
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y[train_index], y[test_index]

                # Apply SMOTE for oversampling to address class imbalance
                X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

                # Apply Random Under Sampling to the majority class
                X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train_resampled, y_train_resampled)

                # Normalize features using StandardScaler
                scaler = StandardScaler()
                X_train_resampled_scaled = scaler.fit_transform(X_train_resampled)
                X_test_scaled = scaler.transform(X_test)

                # Train XGBoost model
                xgb = XGBClassifier(max_depth=max_depth, learning_rate=learning_rate, n_estimators=n_estimators)
                xgb.fit(X_train_resampled_scaled, y_train_resampled)

                # Evaluate model
                y_pred = xgb.predict(X_test_scaled)

                # Calculate evaluation metrics
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, average='weighted')
                recall = recall_score(y_test, y_pred, average='weighted')
                f1 = f1_score(y_test, y_pred, average='weighted')
                conf_matrix = confusion_matrix(y_test, y_pred)

                # Print evaluation metrics and test participant ID
                test_id = participants.iloc[test_index].iloc[0]
                print(f"Test ID: {test_id}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-score: {f1}")

                # Store evaluation metrics
                accuracies.append(accuracy)
                precisions.append(precision)
                recalls.append(recall)
                f1_scores.append(f1)
                conf_matrices.append(conf_matrix)

            # Calculate mean evaluation metrics
            mean_accuracy = sum(accuracies) / len(accuracies)

            # Save best parameters
            if mean_accuracy > best_accuracy:
                best_accuracy = mean_accuracy
                best_params = {'max_depth': max_depth, 'learning_rate': learning_rate, 'n_estimators': n_estimators}

print("Best parameters:", best_params)
print("Best accuracy:", best_accuracy)


Training XGBoost with max_depth=3, learning_rate=0.01, n_estimators=100:
Test ID: 15, Accuracy: 0.7277406073082862, Precision: 0.9020293851759812, Recall: 0.7277406073082862, F1-score: 0.8055657124687493
Test ID: 5C, Accuracy: 0.598487642936186, Precision: 0.6220526008715797, Recall: 0.598487642936186, F1-score: 0.6080527239695745
Test ID: 6B, Accuracy: 0.48985167837626853, Precision: 0.8929551383368798, Recall: 0.48985167837626853, F1-score: 0.5524241627449027
Test ID: 6D, Accuracy: 0.8434712084347121, Precision: 0.7220242108130566, Recall: 0.8434712084347121, F1-score: 0.7780369410550431
Test ID: 7A, Accuracy: 0.6907302038880986, Precision: 0.6468110398328624, Recall: 0.6907302038880986, F1-score: 0.6477962190283598
Test ID: 7E, Accuracy: 0.4035197988686361, Precision: 0.25036968652589536, Recall: 0.4035197988686361, F1-score: 0.30900978775878535
Test ID: 83, Accuracy: 0.733224793580649, Precision: 0.7665721502486013, Recall: 0.733224793580649, F1-score: 0.749385492034822
Test ID: 8B

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test ID: EG, Accuracy: 0.04307334109429569, Precision: 1.0, Recall: 0.04307334109429569, F1-score: 0.08258928571428571
Test ID: F5, Accuracy: 0.7839381720430108, Precision: 0.8636975189214959, Recall: 0.7839381720430108, F1-score: 0.8218873357678278
Training XGBoost with max_depth=3, learning_rate=0.01, n_estimators=200:
Test ID: 15, Accuracy: 0.6963458569222851, Precision: 0.901077120204872, Recall: 0.6963458569222851, F1-score: 0.7851658522077741
Test ID: 5C, Accuracy: 0.5756178531907046, Precision: 0.5649171531467868, Recall: 0.5756178531907046, F1-score: 0.5699411667208174
Test ID: 6B, Accuracy: 0.2576112412177986, Precision: 0.8815381140675844, Recall: 0.2576112412177986, F1-score: 0.2556802142628813
Test ID: 6D, Accuracy: 0.8442822384428224, Precision: 0.7221288115046713, Recall: 0.8442822384428224, F1-score: 0.7784425798600635
Test ID: 7A, Accuracy: 0.6659554291133238, Precision: 0.6231924396031182, Recall: 0.6659554291133238, F1-score: 0.6332177289808699
Test ID: 7E, Accuracy: 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test ID: EG, Accuracy: 0.030558789289871945, Precision: 1.0, Recall: 0.030558789289871945, F1-score: 0.059305280994069474
Test ID: F5, Accuracy: 0.5551075268817204, Precision: 0.8518672660264944, Recall: 0.5551075268817205, F1-score: 0.6668846802213341
Training XGBoost with max_depth=3, learning_rate=0.01, n_estimators=300:
Test ID: 15, Accuracy: 0.6855378281008749, Precision: 0.9022901173128548, Recall: 0.6855378281008749, F1-score: 0.7780079906130707
Test ID: 5C, Accuracy: 0.5719291774253044, Precision: 0.5717061640774178, Recall: 0.5719291774253044, F1-score: 0.5718175210940313
Test ID: 6B, Accuracy: 0.24726775956284153, Precision: 0.8807254215456117, Recall: 0.24726775956284153, F1-score: 0.23943551567123844
Test ID: 6D, Accuracy: 0.8453636117869695, Precision: 0.7222680138739525, Recall: 0.8453636117869695, F1-score: 0.7789828769616228
Test ID: 7A, Accuracy: 0.661688003793267, Precision: 0.6209680728547569, Recall: 0.661688003793267, F1-score: 0.6316451070995521
Test ID: 7E, Accur

...

Test ID: EG, Accuracy: 0.1108847497089639, Precision: 1.0, Recall: 0.1108847497089639, F1-score: 0.19963321980613044
Test ID: F5, Accuracy: 0.6565860215053764, Precision: 0.851084721867472, Recall: 0.6565860215053764, F1-score: 0.7412896137320335
Training XGBoost with max_depth=7, learning_rate=0.1, n_estimators=200:
Test ID: 15, Accuracy: 0.784354091610911, Precision: 0.9283056177413632, Recall: 0.784354091610911, F1-score: 0.844482780450303
Test ID: 5C, Accuracy: 0.5868683142751752, Precision: 0.605981491473587, Recall: 0.5868683142751752, F1-score: 0.5950742726878988
Test ID: 6B, Accuracy: 0.6190476190476191, Precision: 0.8107878308798503, Recall: 0.6190476190476191, F1-score: 0.6803917749984293
Test ID: 6D, Accuracy: 0.7364152473641524, Precision: 0.8823874693231607, Recall: 0.7364152473641524, F1-score: 0.7731815836727791
Test ID: 7A, Accuracy: 0.6735419630156472, Precision: 0.6366018009207577, Recall: 0.6735419630156472, F1-score: 0.6451029400784959
Test ID: 7E, Accuracy: 0.45945

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test ID: EG, Accuracy: 0.179860302677532, Precision: 1.0, Recall: 0.179860302677532, F1-score: 0.30488406512086824
Test ID: F5, Accuracy: 0.6737231182795699, Precision: 0.8530352311974717, Recall: 0.6737231182795699, F1-score: 0.752849403000218
Training XGBoost with max_depth=7, learning_rate=0.1, n_estimators=300:
Test ID: 15, Accuracy: 0.7812660833762224, Precision: 0.9267869110983417, Recall: 0.7812660833762224, F1-score: 0.8423415527166909
Test ID: 5C, Accuracy: 0.5791220951678347, Precision: 0.602503210684365, Recall: 0.5791220951678347, F1-score: 0.5888577404256735
Test ID: 6B, Accuracy: 0.6098750975800156, Precision: 0.8030511695288197, Recall: 0.6098750975800156, F1-score: 0.6727112672655828
Test ID: 6D, Accuracy: 0.7174912138415788, Precision: 0.8776714591055518, Recall: 0.7174912138415788, F1-score: 0.7574206157170534
Test ID: 7A, Accuracy: 0.6686818397344713, Precision: 0.6297496050828735, Recall: 0.6686818397344713, F1-score: 0.6391394293904106
Test ID: 7E, Accuracy: 0.4852

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test ID: EG, Accuracy: 0.23661233993015135, Precision: 1.0, Recall: 0.23661233993015135, F1-score: 0.38267827724170395
Test ID: F5, Accuracy: 0.6848118279569892, Precision: 0.8542498060082031, Recall: 0.6848118279569894, F1-score: 0.7602039558054652
Best parameters: {'max_depth': 7, 'learning_rate': 0.1, 'n_estimators': 300}
Best accuracy: 0.5612822742389483


KNN

In [ ]:
# Replace label values
df['label'] = df['label'].replace({0.0: 'low stress', 1.0: 'low stress', 2.0: 'high stress'})

# Print class counts
class_counts = df['label'].value_counts()
print(class_counts)


high stress    53081
low stress     18402
Name: label, dtype: int64


In [ ]:
import pandas as pd
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE


# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Prepare features and target variable
X = df.drop(columns=['id', 'label'])  # Exclude 'id' and 'label' columns
y = df['label']
participants = df['id']

# Define oversampler (SMOTE)
oversampler = SMOTE(random_state=42)

# Implement leave-one-participant-out cross-validation
logo = LeaveOneGroupOut()

# KNN parameters for tuning
params = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

best_params = None
best_accuracy = 0

# Iterate over different parameter combinations
for n_neighbors in params['n_neighbors']:
    for weights in params['weights']:
        for p in params['p']:
            print(f"Tuning KNN with n_neighbors={n_neighbors}, weights={weights}, p={p}:")

            # Lists to store accuracy for each iteration
            accuracies = []

            for train_index, test_index in logo.split(X, y, groups=participants):
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y[train_index], y[test_index]

                # Apply SMOTE for oversampling to address class imbalance
                X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

                # Normalize features using StandardScaler
                scaler = StandardScaler()
                X_train_resampled_scaled = scaler.fit_transform(X_train_resampled)
                X_test_scaled = scaler.transform(X_test)

                # Train KNN model
                knn = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, p=p)
                knn.fit(X_train_resampled_scaled, y_train_resampled)

                # Evaluate model
                y_pred = knn.predict(X_test_scaled)

                # Calculate accuracy
                accuracy = accuracy_score(y_test, y_pred)

                # Store accuracy for each iteration
                accuracies.append(accuracy)

            # Calculate mean accuracy for current parameter combination
            mean_accuracy = sum(accuracies) / len(accuracies)
            print(f"Mean Accuracy: {mean_accuracy}")

            # Save best parameters
            if mean_accuracy > best_accuracy:
                best_accuracy = mean_accuracy
                best_params = {'n_neighbors': n_neighbors, 'weights': weights, 'p': p}

print("Best parameters:", best_params)
print("Best accuracy:", best_accuracy)


Tuning KNN with n_neighbors=3, weights=uniform, p=1:
Mean Accuracy: 0.5276802014997548
Tuning KNN with n_neighbors=3, weights=uniform, p=2:
Mean Accuracy: 0.5251555228615005
Tuning KNN with n_neighbors=3, weights=distance, p=1:
Mean Accuracy: 0.5277310862575271
Tuning KNN with n_neighbors=3, weights=distance, p=2:
Mean Accuracy: 0.5251143585021238
Tuning KNN with n_neighbors=5, weights=uniform, p=1:
Mean Accuracy: 0.5187308412800442
Tuning KNN with n_neighbors=5, weights=uniform, p=2:
Mean Accuracy: 0.5159948025980079
Tuning KNN with n_neighbors=5, weights=distance, p=1:
Mean Accuracy: 0.5190395890602619
Tuning KNN with n_neighbors=5, weights=distance, p=2:
Mean Accuracy: 0.5159894567236095
Tuning KNN with n_neighbors=7, weights=uniform, p=1:
Mean Accuracy: 0.510814724358096
Tuning KNN with n_neighbors=7, weights=uniform, p=2:
Mean Accuracy: 0.5072149176414887
Tuning KNN with n_neighbors=7, weights=distance, p=1:
Mean Accuracy: 0.511721965170768
Tuning KNN with n_neighbors=7, weights=d

In [ ]:
df


,id,label,avg_X,max_X,min_X,X_median,avg_Y,max_Y,min_Y,Y_median,...,min_EDA,EDA_median,avg_HR,max_HR,min_HR,HR_median,avg_TEMP,max_TEMP,min_TEMP,TEMP_median
0,15,low stress,19.21875,7.0,1.0,19.0,-42.50625,-64.0,-27.0,-43.0,...,3.801695,3.976439,103.190,103.27,103.07,103.20,33.876,33.95,33.81,33.87
1,15,low stress,17.93750,22.0,15.0,18.0,-43.73750,-45.0,-41.0,-44.0,...,3.547946,3.619714,103.188,103.30,103.00,103.25,33.966,33.99,33.93,33.97
2,15,low stress,-27.60625,-37.0,-20.0,-27.0,9.96250,9.0,-1.0,10.0,...,4.737687,4.808173,109.362,110.00,108.68,109.38,34.110,34.13,34.09,34.11
3,15,low stress,-28.59375,-38.0,-17.0,-29.0,10.63750,9.0,-1.0,10.0,...,4.615938,4.645414,112.220,112.35,111.90,112.28,34.148,34.16,34.13,34.15
4,15,low stress,-50.07500,-99.0,-13.0,-48.0,-29.84375,6.0,-1.0,-29.0,...,4.038785,4.157969,99.768,99.92,99.62,99.77,34.336,34.37,34.31,34.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71478,F5,high stress,-32.43125,-9.0,-10.0,-20.0,-13.86875,-9.0,-10.0,-14.0,...,3.378974,3.728609,80.550,81.05,80.05,80.57,34.590,34.61,34.57,34.59
71479,F5,high stress,8.08750,9.0,0.0,9.0,60.28750,62.0,57.0,61.0,...,3.706447,3.848606,87.034,87.08,86.97,87.05,34.594,34.61,34.57,34.59
71480,F5,high stress,-22.76250,9.0,-1.0,-20.0,-18.54375,8.0,-1.0,-7.0,...,10.031460,10.031460,92.774,92.82,92.70,92.80,34.322,34.34,34.29,34.33
71481,F5,high stress,-11.18125,-9.0,-10.0,-11.0,-61.88750,-65.0,-58.0,-62.0,...,8.944329,8.968663,91.470,91.55,91.40,91.47,34.160,34.18,34.15,34.16


Random Forst

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut, cross_val_predict
from sklearn.metrics import classification_report
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df[df['label'] == 'high stress']
df_minority = df[df['label'] == 'low stress']

# Upsample minority class
df_minority_upsampled = resample(df_minority,
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=42) # reproducible results

# Combine majority class with upsampled minority class
df_balanced = pd.concat([df_majority, df_minority_upsampled])

# Extract features and target variable from balanced dataset
X_balanced = df_balanced.drop(['id', 'label'], axis=1)
y_balanced = df_balanced['label']

# Initialize Random Forest classifier with hyperparameter tuning
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, class_weight='balanced')

# Define leave-one-group-out cross-validation
logo = LeaveOneGroupOut()

# Perform leave-one-group-out cross-validation on balanced data
y_pred_loo_balanced = cross_val_predict(rf_classifier, X_balanced, y_balanced, groups=df_balanced['id'], cv=logo)

# Evaluate classification report after leave-one-group-out cross-validation
print("Classification Report After Leave one out cross-validation on balanced data:")
print(classification_report(y_balanced, y_pred_loo_balanced))

# Train the model on the entire balanced dataset
rf_classifier.fit(X_balanced, y_balanced)

# Predict stress levels using the trained model
y_pred_balanced = rf_classifier.predict(X_balanced)

# Evaluate classification report after training on entire balanced dataset
print("\nClassification Report after training on entire balanced dataset:")
print(classification_report(y_balanced, y_pred_balanced))


Classification Report After Leave one out cross-validation on balanced data:
              precision    recall  f1-score   support

 high stress       0.53      0.68      0.60     53081
  low stress       0.56      0.41      0.47     53081

    accuracy                           0.54    106162
   macro avg       0.55      0.54      0.53    106162
weighted avg       0.55      0.54      0.53    106162


Classification Report after training on entire balanced dataset:
              precision    recall  f1-score   support

 high stress       0.83      0.89      0.86     53081
  low stress       0.88      0.81      0.85     53081

    accuracy                           0.85    106162
   macro avg       0.85      0.85      0.85    106162
weighted avg       0.85      0.85      0.85    106162



In [ ]:
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut, cross_val_predict
from sklearn.metrics import classification_report
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df[df['label'] == 'high stress']
df_minority = df[df['label'] == 'low stress']

# Upsample minority class
df_minority_upsampled = resample(df_minority,
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=42) # reproducible results

# Combine majority class with upsampled minority class
df_balanced = pd.concat([df_majority, df_minority_upsampled])

# Extract features and target variable from balanced dataset
X_balanced = df_balanced.drop(['id', 'label'], axis=1)
y_balanced = df_balanced['label']

# Apply PCA to the features
pca = PCA(n_components=0.80)  # Choose number of components to explain 95% of variance
X_pca = pca.fit_transform(X_balanced)

# Initialize Random Forest classifier with hyperparameter tuning
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, class_weight='balanced')

# Define leave-one-group-out cross-validation
logo = LeaveOneGroupOut()

# Perform leave-one-group-out cross-validation on PCA-transformed data
y_pred_loo_pca = cross_val_predict(rf_classifier, X_pca, y_balanced, groups=df_balanced['id'], cv=logo)

# Evaluate classification report after leave-one-group-out cross-validation on PCA-transformed data
print("Classification Report After Leave one out cross-validation on PCA-transformed data:")
print(classification_report(y_balanced, y_pred_loo_pca))


Classification Report After Leave one out cross-validation on PCA-transformed data:
              precision    recall  f1-score   support

 high stress       0.48      0.59      0.53     53081
  low stress       0.47      0.37      0.42     53081

    accuracy                           0.48    106162
   macro avg       0.48      0.48      0.47    106162
weighted avg       0.48      0.48      0.47    106162

